In [2]:
import vectorbt as vbt

price = vbt.YFData.download('SOL-USD').get('Close')
price

Date
2020-04-10 00:00:00+00:00      0.951054
2020-04-11 00:00:00+00:00      0.776819
2020-04-12 00:00:00+00:00      0.882507
2020-04-13 00:00:00+00:00      0.777832
2020-04-14 00:00:00+00:00      0.661925
                                ...    
2025-05-20 00:00:00+00:00    168.483688
2025-05-21 00:00:00+00:00    173.681732
2025-05-22 00:00:00+00:00    179.690948
2025-05-23 00:00:00+00:00    173.825424
2025-05-24 00:00:00+00:00    175.244125
Freq: D, Name: Close, Length: 1871, dtype: float64

# Fetch Today's BTC-USD 00:00 UTC Hourly Candle
This cell fetches the opening (00:00 UTC) hourly candle for BTC-USD for today using vectorbt and displays its OHLC values.

In [8]:
import vectorbt as vbt
import pandas as pd
from datetime import datetime, timedelta

# Get today's date in UTC as string
utc_today = datetime.utcnow().date()
start_str = str(utc_today)
end_str = str(utc_today + timedelta(days=1))

# Download 1-hour BTC-USD data for today
btc_1h = vbt.YFData.download(
    'BTC-USD',
    start=start_str,
    end=end_str,
    interval='1h'
)

# Extract OHLC data
open_ = btc_1h.get('Open')
high_ = btc_1h.get('High')
low_ = btc_1h.get('Low')
close_ = btc_1h.get('Close')

# Filter for 00:00 UTC candle
mask = (open_.index.hour == 0)
df=0
if mask.any():
    df = pd.DataFrame({
        'Open': open_[mask],
        'High': high_[mask],
        'Low': low_[mask],
        'Close': close_[mask],
    })
    print("BTC-USD 00:00 UTC Candle for Today:")
    # print(df)
else:
    print("No 00:00 UTC candle found for today.")

df

BTC-USD 00:00 UTC Candle for Today:


,Open,High,Low,Close
Datetime,,,,
2025-05-25 00:00:00+00:00,107789.9375,108105.921875,107357.328125,107457.78125


# Simulate Trade Based on 00:00 UTC Candle and 5-Minute Candles
This cell determines trade direction (long/short) from the 00:00 UTC candle, sets SL/TP, and simulates trade outcome by scanning 5-minute candles until SL or TP is hit.

In [12]:
# Determine trade direction
if df is not None and not isinstance(df, int) and not df.empty:
    entry_row = df.iloc[0]
    entry_time = df.index[0]
    entry_price = entry_row['Close']
    direction = 'Long' if entry_row['Close'] > entry_row['Open'] else 'Short'
    sl = entry_price * (1 - 0.005) if direction == 'Long' else entry_price * (1 + 0.005)
    tp = entry_price * (1 + 0.01) if direction == 'Long' else entry_price * (1 - 0.01)
    
    # Download 5-minute BTC-USD data for today
    btc_5m = vbt.YFData.download(
        'BTC-USD',
        start=start_str,
        end=end_str,
        interval='5m'
    )
    high_5m = btc_5m.get('High')
    low_5m = btc_5m.get('Low')
    # Only consider candles after entry_time
    mask_5m = high_5m.index > entry_time
    high_5m = high_5m[mask_5m]
    low_5m = low_5m[mask_5m]
    exit_price = None
    exit_time = None
    result = None
    ohlc_hit = None  # Store the OHLC of the candle where SL/TP is hit
    for idx in high_5m.index:
        high = high_5m.loc[idx]
        low = low_5m.loc[idx]
        open5 = btc_5m.get('Open').loc[idx]
        close5 = btc_5m.get('Close').loc[idx]
        if direction == 'Long':
            if low <= sl:
                exit_price = sl
                exit_time = idx
                result = 'Loss'
                ohlc_hit = {'Open': open5, 'High': high, 'Low': low, 'Close': close5, 'Time': idx}
                break
            if high >= tp:
                exit_price = tp
                exit_time = idx
                result = 'Profit'
                ohlc_hit = {'Open': open5, 'High': high, 'Low': low, 'Close': close5, 'Time': idx}
                break
        else:
            if high >= sl:
                exit_price = sl
                exit_time = idx
                result = 'Loss'
                ohlc_hit = {'Open': open5, 'High': high, 'Low': low, 'Close': close5, 'Time': idx}
                break
            if low <= tp:
                exit_price = tp
                exit_time = idx
                result = 'Profit'
                ohlc_hit = {'Open': open5, 'High': high, 'Low': low, 'Close': close5, 'Time': idx}
                break
    if exit_price is not None:
        # Calculate PnL in USD based on $1000 position size, 1x leverage
        position_size = 1000  # USDT
        if direction == 'Long':
            pnl_pct = (exit_price - entry_price) / entry_price
        else:
            pnl_pct = (entry_price - exit_price) / entry_price
        pnl_usd = pnl_pct * position_size
        print(f"Direction: {direction}")
        print(f"Entry Price: {entry_price:.2f}")
        print(f"SL: {sl:.2f}, TP: {tp:.2f}")
        print(f"Exit Time: {exit_time}")
        print(f"Exit Price: {exit_price:.2f}")
        print(f"Result: {result}")
        print(f"PnL: {pnl_usd:.2f} USDT (Return: {pnl_pct*100:.2f}%)")
        if ohlc_hit is not None:
            ohlc_df = pd.DataFrame([ohlc_hit])
            print("\nOHLC of the 5m candle where SL/TP was hit:")
            print(ohlc_df)
    else:
        print("Neither SL nor TP was hit in today's 5-minute candles after entry.")
else:
    print("No valid 00:00 UTC candle to base the trade on.")

Direction: Short
Entry Price: 107457.78
SL: 107995.07, TP: 106383.20
Exit Time: 2025-05-25 00:10:00+00:00
Exit Price: 107995.07
Result: Loss
PnL: -5.00 USDT (Return: -0.50%)

OHLC of the 5m candle where SL/TP was hit:
            Open           High            Low          Close  \
0  108005.085938  108035.320312  107989.203125  107989.203125   

                       Time  
0 2025-05-25 00:10:00+00:00  
